# Clustering Template

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

randomState=42
np.random.seed(randomState)

## Esplore Data

In [ ]:
url = ''
df = pd.read_csv(url) #delimiter=, index_col=, names=

print(f' Data frame has {df.shape[0]} samples, and {df.shape[1]-1} features ')

In [ ]:
# Check the first 5 rows of the dataset
df.head()

In [ ]:
# explore the distribution of the target variable
# count help to see if there are some missing values
df.describe()

In [ ]:
# n rows with missing values
df.shape[0]-df.dropna().shape[0]

In [ ]:
# Count the number of missing values per columns
df.isna().sum()

In [ ]:
# visualize the distribution of the features
# check for outliers and different scales of the features
df.boxplot(figsize=(15,10))
plt.show()

In [ ]:
# visualize linear relationship between features
sns.pairplot(df)
plt.show()

In [ ]:
# is some feature have different value for every row/sample we eliminate it
df['Territorio'].unique().shape

#### or ####

df.nunique()

## Preprocessing

In [ ]:
print(f'there are {df.isna().sum().sum()} null values')
df1 = df.copy().dropna()
print(f'there are {df1.isna().sum().sum()} null values')
print(f'Data frame has {df1.shape[0]} samples, and {df1.shape[1]} features')

In [ ]:
# (OPTIONAL) If there is a string variable, we need to encode it to numerical values 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
column_to_transform = ''
transformed_column = le.fit_transform(df1[column_to_transform].values)
df1[column_to_transform] = transformed_column

In [ ]:
# (OPTIONAL) use this to convert nominal labels to numerical values
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
column_to_transform = 'exemple_column'
enc_data = one.fit_transform(df[column_to_transform].values)
l = list(one.categories_[0])
enc_df = pd.DataFrame(enc_data.toarray(),columns=l)
df = df.join(enc_df)
df = df.drop([column_to_transform],axis=1)
df.head()

In [ ]:
# (OPTIONAL) use this to convert ordinal labels to numerical values
from sklearn.preprocessing import OrdinalEncoder
categories = ['bad','good','very good'] # exemple of ordinal categories
oe = OrdinalEncoder(categories=categories,dtype=int)
column_to_transform = 'col_name'
df[column_to_transform] = oe.fit_transform(df[column_to_transform].values)


In [ ]:
# Change the ranges of the features to be between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_processed = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

In [ ]:
# Data standardization
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.pipeline import make_pipeline
preprocessor = make_pipeline(PowerTransformer(),StandardScaler())
df_processed = pd.DataFrame(preprocessor.fit_transform(df),columns=df.columns)

In [ ]:
# (OPTIONAL) DO THIS STEP ONLY IF THE DATASET HAS A LARGE NUMBER OF FEATURES (E.G. MORE THAN 20)
# remove features with low variance, or with high correlation with other features

from sklearn.decomposition import PCA
pca = PCA()
df_tranformed = pca.fit_transform(df)
print(f'Explained variance ration: {pca.explained_variance_ration_}')
min_variance = 0.9 # or 0.8
variance_cumsum = np.cumsum(pca.explained_variance_ration_.copy())
cutoff_index = np.argmax(variance_cumsum>min_variance)
df = df_tranformed[:,:cutoff_index+1]
print(f'df shape after PCA {df.shape}')

## Train

In [ ]:
X = df
n_clusters = [*range(2,11)]

### Kmeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score

param_km = [{'n_clusters':n_clusters}]
pg = list(ParameterGrid(param_km))
report_km = pd.DataFrame([],columns=['n clusters','inertia','silhouette_score'])

for param in pg:
    km = KMeans(n_clusters=param['n_clusters'],random_state=randomState)
    y_km = km.fit_predict(X)
    report_km.loc[len(report_km)] = [
        param['n_clusters'],
        km.inertia_,
        silhouette_score(X,y_km)
    ]

display(report_km)

#### Checking for Hellbows - best solution

In [ ]:
fix,ax = plt.subplots()
ax.plot(n_clusters,report_km['inertia'],color='red')
ax.set_xlabel('Number of clusters')
ax.set_ylabel('inertia',color='red')

ax2 = ax.twinx()
ax2.plot(n_clusters,report_km['silhouette_score'],color='blue')
ax2.set_ylabel('silhouette_score',color='blue')
ax2.set_ylim(0,1)

plt.show()

# Hellbows are the optimal number of clusters.
# hellbows are where the inertia has a huge decrease and where the silhuette has a maximun

### Aglomerative

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

n_clusters = [*range(2,7)]
param_ac = [{
    'n_clusters': n_clusters,
    'linkage': ['ward','complete','avarage','single']
}]
pg = list(ParameterGrid(param_ac))
result_ac = pd.DataFrame([],columns=['n_clusters','linkage','silhouette_score'])

for param in pg:
    ac = AgglomerativeClustering(
        n_clusters=param['n_clusters'],
        linkage=param['linkage']
    )
    y_ac = ac.fit_predict(X)
    result_ac.loc[len(result_ac)]=[
        param['n_clusters'],
        param['linkage'],
        silhouette_score(X,y_ac)
    ]

display(result_ac.sort_values(by='silhouette_score', ascending=False))

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

param_grid = {'eps': list(np.arange(0.001, 1, 0.005)), 'min_samples': list(range(2,10,1))}
pg = list(ParameterGrid(param_grid))

result_dbs = pd.DataFrame([],columns=['n_clusters','eps','min_samples','silhouette_score','unclust%'])

for param in pg:
    dbs = DBSCAN(eps=param['eps'],min_samples=param['min_samples'])
    y_dbs = dbs.fit_predict(X)
    cluster_labels_all = np.unique(y_dbs)
    cluster_labels = cluster_labels_all[cluster_labels_all != -1] # -1 is the noise
    n_clusters = len(cluster_labels)
    if n_clusters > 1 and n_clusters < len(X):
        # remove noise form the silhouette calcolus
        X_cl = X.loc[y_dbs!=-1,:]
        y_dbs_cl = y_dbs[y_dbs!=-1]
        sil_score = silhouette_score(X_cl, y_dbs_cl)
        uncl_p = (1 - y_dbs_cl.shape[0]/y_dbs.shape[0]) * 100 # percentage of unclustered points
        result_dbs.loc[len(result_dbs)]=[
            n_clusters,
            param['eps'],
            param['min_samples'],
            sil_score,
            uncl_p
        ]


##################### OR ############################
param_dbs = [{
    'eps': [*range(0.001,1,0.05)],
    'min_samples': [*range(2,10)]
}]
pg_dbs = ParameterGrid(param_dbs)
report_dbs = pd.DataFrame([],columns=['n_clusters','eps','min_samples','silouette_score','unclust%'])

for param in pg_dbs:
    dbs = DBSCAN(eps=param['eps'],min_samples=['min_samples'])
    y_dbs = dbs.fit_predict(X)
    y_dbs_clustered = y_dbs[y_dbs != -1, :]
    X_dbs_clustered = X.loc[y_dbs != -1, :]
    n_cluster = len(np.unique(y_dbs_clustered))
    unclust = 1 - y_dbs_clustered.shape[0]/y_dbs.shape[0]
    if n_cluster > 1 and n_cluster < len(X):
        report_dbs.loc[len(report_dbs)]=[
            n_cluster,
            param['eps'],
            param['min_samples'],
            silhouette_score(X_dbs_clustered, y_dbs_clustered),
            unclust*100
        ]

## Display Data

In [ ]:
# Pie chart of the cluster sizes
clust_sizes_km = np.unique(y_km,return_counts=True)
pd.DataFrame(clust_sizes_km[1]).plot.pie(y=0, autopct='%1.1f%%')
plt.show()

In [ ]:
# Compare clusters and features
X['cluster_km']=y_km
sns.pairplot(data=X, hue='cluster_km');
plt.show()

In [ ]:
# Visualize DBSCAN results (the professor parameter generate 200 estimators, these is to show only the bests)

sil_thr = 0.0  # visualize results only for combinations with silhouette above the threshold
unc_thr = 22 # visualize results only for combinations with unclustered% below the threshold
n_clu_max_thr = 5
result_dbs[(result_dbs['silhouette']>=sil_thr)\
         & (result_dbs['unclust%']<=unc_thr)\
         & (result_dbs['n_clusters']<=n_clu_max_thr)].sort_values(['silhouette','unclust%'],ascending=[False,True])